In [2]:
import nilearn
from nilearn import datasets, plotting
import os
import requests
import csv
import pandas as pd
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt

In [13]:
msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords

whole_data = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-1mm', data_dir='/mfs/io/groups/dmello/projects/dynamric/tree_mri/code/HarvardOxford')
whole_coords = nilearn.plotting.find_parcellation_cut_coords(whole_data.maps)

[get_dataset_dir] Dataset found in /home/dxj180007/nilearn_data/msdl_atlas
[get_dataset_dir] Dataset found in /mfs/io/groups/dmello/projects/dynamric/tree_mri/code/HarvardOxford/fsl


In [ ]:
# load 3d shapes
cort_shape = np.loadtxt('abide_30_msdl_pooled_3d_shape.csv', delimiter=',').astype(int)
 # cereb_shape = np.loadtxt('abide_30_diedrichsen_pooled_3d_shape.csv', delimiter=',').astype(int)
whole_shape = np.loadtxt('abide_30_harvard_oxford_pooled_3d_shape.csv', delimiter=',').astype(int)

# load and reshape pooled time series
cort_pooled = np.loadtxt('abide_30_msdl_pooled.csv', delimiter=',').reshape(cort_shape)
# cereb_pooled = np.loadtxt('abide_30_diedrichsen_pooled.csv', delimiter=',').reshape(cereb_shape)
whole_pooled = np.loadtxt('abide_30_harvard_oxford_pooled.csv', delimiter=',').reshape(whole_shape)

In [4]:
print(cort_pooled.shape)  # check shapes of loaded arrays
print(whole_pooled.shape) # check shapes of loaded arrays

(30, 195, 39)
(30, 195, 21)


In [ ]:
# Base URL for the dataset
base_url = "/mfs/io/groups/dmello/projects/dynamric/tree_mri/datasets/abide/preprocessed_dataset/Outputs/dparsf/filt_noglobal/func_preproc"

# Function to construct the download URL
def construct_url(base_url, file_id):
    return f"{base_url}/{file_id}_func_preproc.nii.gz"

s3_pheno_file = "/mfs/io/groups/dmello/projects/dynamric/tree_mri/datasets/abide/Phenotypic_V1_0b_preprocessed1.csv"
pheno_data = pd.read_csv(s3_pheno_file)

# code phenotypes, 0 as typically developing control (TDC), 1 as autism diagnosis
abide_files = os.listdir(base_url)
len(abide_files)

abide_ids = []
phenotype = []

for i in range(len(pheno_data)):
    file_id = pheno_data['FILE_ID'][i]
    dx = pheno_data['DX_GROUP'][i]
    url = f'{file_id}_func_preproc.nii.gz'
    if url in abide_files: 
        abide_ids.append(file_id)
        if dx == 1: phenotype.append(1) # autism diagnosis
        else: phenotype.append(0) # tdc

## simple correlation matrices

In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(
    kind="correlation",
    standardize="zscore_sample",
)

In [ ]:
# separate timeseries for asd and tdc
def get_groups(pooled_subjects, phenotype):
    asd = []
    tdc = []
    for i in range(30):
        if phenotype[i] == 1:
            asd.append(pooled_subjects[i])
        else: tdc.append(pooled_subjects[i])
    asd = np.array(asd)
    tdc = np.array(tdc)

    return {'asd': asd, 'tdc': tdc}

In [ ]:
cort_groups = get_groups(cort_pooled, phenotype)
# cereb_groups = get_groups(cereb_pooled, phenotype)
whole_groups = get_groups(whole_pooled, phenotype)

In [ ]:
# calculate mean correlation matrix and mean corr arr
def get_mean_corr(pooled_subjects: dict, correlation_measure=correlation_measure):

    asd = pooled_subjects['asd']
    tdc = pooled_subjects['tdc']

    correlation_asd = correlation_measure.fit_transform(asd)
    mean_correlation_matrix_asd = correlation_measure.mean_

    correlation_tdc = correlation_measure.fit_transform(tdc)
    mean_correlation_matrix_tdc = correlation_measure.mean_

    mean_corr_asd = np.mean(mean_correlation_matrix_asd, axis=1)
    mean_corr_tdc = np.mean(mean_correlation_matrix_tdc, axis=1)
    
    # dictionary containing tuple with mean correlation matrix and mean correlation
    return {'asd': (correlation_asd, mean_correlation_matrix_asd, mean_corr_asd), 'tdc': (correlation_tdc, mean_correlation_matrix_tdc, mean_corr_tdc)}

In [8]:
#visualize asd and tdc corr side by side
def visualize_corr(asd_mat, tdc_mat):

    # Set the figure size and create subplots
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Wider figure for better readability

    # Set consistent color scale across both plots
    vmin = min(asd_mat.min(), tdc_mat.min())
    vmax = max(asd_mat.max(), tdc_mat.max())

    # Plot the first matrix
    im1 = axes[0].matshow(asd_mat, vmin=vmin, vmax=vmax, cmap='viridis')
    axes[0].set_title('ASD Cort')

    # Plot the second matrix
    im2 = axes[1].matshow(tdc_mat, vmin=vmin, vmax=vmax, cmap='viridis')
    axes[1].set_title('TDC Cort')

    # Add a shared colorbar without overlap
    cbar = fig.colorbar(im1, ax=axes.ravel().tolist(), orientation='vertical', fraction=0.05, pad=0.05)
    cbar.ax.set_position([1, 0.15, 0.02, 0.7])  # [x, y, width, height]

    # Improve layout spacing
    plt.subplots_adjust(wspace=0.3, right=0.9)

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
# get cort corr matrices
mean_corr_cort = get_mean_corr(cort_groups)

# get cereb corr matrices
# mean_corr_cereb = get_mean_corr(cereb_groups)  

# get whole corr matrices
mean_corr_whole = get_mean_corr(whole_groups)

NameError: name 'cort_groups' is not defined

In [ ]:
#visualize asd and tdc corr side by side
def visualize_corr(mean_corr):

    asd_mat = mean_corr['asd'][1]  # mean correlation matrix for ASD
    tdc_mat = mean_corr['tdc'][1]  # mean correlation matrix for TDC

    # Set the figure size and create subplots
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Wider figure for better readability

    # Set consistent color scale across both plots
    vmin = min(asd_mat.min(), tdc_mat.min())
    vmax = max(asd_mat.max(), tdc_mat.max())

    # Plot the first matrix
    im1 = axes[0].matshow(asd_mat, vmin=vmin, vmax=vmax, cmap='viridis')
    axes[0].set_title('ASD Cort')

    # Plot the second matrix
    im2 = axes[1].matshow(tdc_mat, vmin=vmin, vmax=vmax, cmap='viridis')
    axes[1].set_title('TDC Cort')

    # Add a shared colorbar without overlap
    cbar = fig.colorbar(im1, ax=axes.ravel().tolist(), orientation='vertical', fraction=0.05, pad=0.05)
    cbar.ax.set_position([1, 0.15, 0.02, 0.7])  # [x, y, width, height]

    # Improve layout spacing
    plt.subplots_adjust(wspace=0.3, right=0.9)

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_corr(mean_corr_cort) # visualize cort mean correlation matrices

In [ ]:
visualize_corr(mean_corr_whole) # visualize whole mean correlation matrices

In [ ]:
def plot_connectome(mean_correlation_matrix, coords, title='connectome'):
    """
    Plot the connectome using the mean correlation matrix and coordinates.
    """
    # Plotting the connectome
    plotting.plot_connectome(
        mean_correlation_matrix,
        coords,
        title=title,
        edge_threshold="90%",  # Only plot edges above the 90th percentile
        display_mode="lateral",
        colorbar=True,
        node_size=10,
    )
    plt.show()

In [ ]:
plot_connectome(
    mean_corr_cort['asd'][1],  # Mean correlation matrix for ASD
    msdl_coords,  # Coordinates for MSDL atlas
    title = "ASD Cort Connectome"
)


In [ ]:
plot_connectome(
    mean_corr_cort['tdc'][1],  # Mean correlation matrix for TDC
    msdl_coords,  # Coordinates for MSDL atlas
    title = "TDC Cort Connectome"
)

In [ ]:
plot_connectome(
    mean_corr_whole['asd'][1],  # Mean correlation matrix for ASD
    whole_coords,  # Coordinates for Harvard-Oxford atlas
    "ASD Whole Connectome"
)

In [ ]:
plot_connectome(
    mean_corr_whole['tdc'][1],  # Mean correlation matrix for TDC
    whole_coords,  # Coordinates for Harvard-Oxford atlas
    "TDC Whole Connectome"
)

## partial correlation

In [ ]:
partial_correlation_measure = ConnectivityMeasure(
    kind="partial correlation",
    standardize="zscore_sample",
)

In [ ]:
# get cort corr matrices
mean_partial_cort = get_mean_corr(cort_groups, correlation_measure=partial_correlation_measure)

# get cereb corr matrices
# mean_corr_cereb = get_mean_corr(cereb_groups)  

# get whole corr matrices
mean_partial_whole = get_mean_corr(whole_groups, correlation_measure=partial_correlation_measure)

## tangent measure

In [ ]:
tangent_measure = ConnectivityMeasure(
    kind="tangent",
    standardize="zscore_sample",
)

In [ ]:
# get cort corr matrices
mean_tangent_cort = get_mean_corr(cort_groups, correlation_measure=partial_correlation_measure)

# get cereb corr matrices
# mean_corr_cereb = get_mean_corr(cereb_groups)  

# get whole corr matrices
mean_tangent_whole = get_mean_corr(whole_groups, correlation_measure=partial_correlation_measure)